In [ ]:
"""
Base Agent Framework for Multi-Agent System
- Defines core agent structure
- Implements Model Context Protocol (MCP)
- Handles communication between agents
"""

import os
import json
import asyncio
import logging
from typing import Dict, List, Optional, Any, Set, Tuple
from enum import Enum, auto
from dataclasses import dataclass, field
from datetime import datetime
from collections import deque
import uuid
from pydantic import BaseModel, Field

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# ====== Context Definitions (MCP) ======

class ContextType(Enum):
    """Types of contexts that can be shared between agents"""
    SYSTEM = auto()       # System-wide information
    TASK = auto()         # Task-related information
    COMMAND = auto()      # Command execution details
    LEARNING = auto()     # Learning-related information
    THOUGHT = auto()      # Internal thought processes
    MESSAGE = auto()      # Direct messages between agents

class Context(BaseModel):
    """Base context class using Model Context Protocol"""
    context_id: str = Field(default_factory=lambda: str(uuid.uuid4()))
    context_type: ContextType
    created_at: datetime = Field(default_factory=datetime.now)
    source_agent: str
    target_agents: List[str] = Field(default_factory=list)
    priority: int = 1  # 1-10, higher is more important
    metadata: Dict[str, Any] = Field(default_factory=dict)
    content: Dict[str, Any] = Field(default_factory=dict)
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert context to dictionary for transmission"""
        return {
            "context_id": self.context_id,
            "context_type": self.context_type.name,
            "created_at": self.created_at.isoformat(),
            "source_agent": self.source_agent,
            "target_agents": self.target_agents,
            "priority": self.priority,
            "metadata": self.metadata,
            "content": self.content
        }

# ====== Thought Process ======

class ThoughtType(Enum):
    """Types of thoughts an agent can have"""
    OBSERVATION = auto()  # Raw observations
    ANALYSIS = auto()     # Analysis of observations
    PLAN = auto()         # Planning future actions
    DECISION = auto()     # Making decisions
    LEARNING = auto()     # Learning from experiences
    REFLECTION = auto()   # Reflecting on past actions

class Thought(BaseModel):
    """Represents an internal thought process of an agent"""
    thought_id: str = Field(default_factory=lambda: str(uuid.uuid4()))
    thought_type: ThoughtType
    created_at: datetime = Field(default_factory=datetime.now)
    content: str
    confidence: float = 0.5  # 0.0-1.0, higher is more confident
    references: List[str] = Field(default_factory=list)  # IDs of related thoughts
    metadata: Dict[str, Any] = Field(default_factory=dict)

# ====== Memory System ======

class Memory(BaseModel):
    """Base memory class for agents"""
    memories: Dict[str, Thought] = Field(default_factory=dict)
    short_term: List[str] = Field(default_factory=list)  # Recent thought IDs
    long_term: Dict[str, List[str]] = Field(default_factory=dict)  # Categorized thought IDs
    patterns: Dict[str, Dict[str, float]] = Field(default_factory=dict)  # Pattern: {thought_id: confidence}
    
    def add_thought(self, thought: Thought) -> None:
        """Add a thought to memory"""
        self.memories[thought.thought_id] = thought
        self.short_term.append(thought.thought_id)
        
        # Limit short-term memory size
        if len(self.short_term) > 50:
            self.short_term.pop(0)
        
        # Categorize in long-term memory
        category = thought.thought_type.name
        if category not in self.long_term:
            self.long_term[category] = []
        self.long_term[category].append(thought.thought_id)
    
    def get_relevant_thoughts(self, query: str, thought_types: Optional[List[ThoughtType]] = None, 
                             limit: int = 5) -> List[Thought]:
        """Retrieve relevant thoughts based on a query"""
        # Simple relevance scoring based on content matching
        # In production, use vector embeddings or more sophisticated retrieval
        relevant_thoughts = []
        
        for thought_id, thought in self.memories.items():
            # Filter by thought type if specified
            if thought_types and thought.thought_type not in thought_types:
                continue
                
            # Simple relevance check (to be replaced with better matching)
            relevance = 0
            for word in query.lower().split():
                if word in thought.content.lower():
                    relevance += 1
            
            if relevance > 0:
                relevant_thoughts.append((thought, relevance))
        
        # Sort by relevance and return top results
        relevant_thoughts.sort(key=lambda x: x[1], reverse=True)
        return [t[0] for t in relevant_thoughts[:limit]]

# ====== Base Agent ======

class AgentRole(Enum):
    """Roles that agents can have in the system"""
    ARCHITECT = auto()
    PROJECT_MANAGER = auto()
    PROGRAMMER = auto()
    SPECIALIST = auto()

class AgentStatus(Enum):
    """Possible statuses of an agent"""
    INITIALIZING = auto()
    READY = auto()
    BUSY = auto()
    LEARNING = auto()
    OFFLINE = auto()

class BaseAgent:
    """Base class for all agents in the system"""
    
    def __init__(self, agent_id: str, role: AgentRole, model_path: str):
        self.agent_id = agent_id
        self.role = role
        self.model_path = model_path
        self.status = AgentStatus.INITIALIZING
        self.memory = Memory()
        self.context_queue = asyncio.Queue()
        self.subscribed_contexts: Set[ContextType] = set()
        self.model = None  # To be loaded by subclasses
        self.logger = logging.getLogger(f"agent.{agent_id}")
        
    async def initialize(self) -> None:
        """Initialize the agent and load model"""
        self.logger.info(f"Initializing agent {self.agent_id}")
        # Load model - to be implemented by subclasses
        await self._load_model()
        self.status = AgentStatus.READY
        
    async def _load_model(self) -> None:
        """Load the language model - to be implemented by subclasses"""
        raise NotImplementedError("Subclasses must implement _load_model")
    
    def subscribe_to_context(self, context_type: ContextType) -> None:
        """Subscribe to a specific context type"""
        self.subscribed_contexts.add(context_type)
    
    async def receive_context(self, context: Context) -> None:
        """Receive a context and add it to the queue"""
        if context.context_type in self.subscribed_contexts:
            await self.context_queue.put(context)
    
    async def send_context(self, context: Context, message_broker) -> None:
        """Send a context through the message broker"""
        context.source_agent = self.agent_id
        await message_broker.publish_context(context)
    
    async def process_contexts(self, message_broker) -> None:
        """Process contexts from the queue"""
        while True:
            context = await self.context_queue.get()
            self.logger.debug(f"Processing context: {context.context_id}")
            
            # Process based on context type
            if context.context_type == ContextType.MESSAGE:
                await self._process_message(context)
            elif context.context_type == ContextType.TASK:
                await self._process_task(context)
            elif context.context_type == ContextType.COMMAND:
                await self._process_command(context)
            elif context.context_type == ContextType.LEARNING:
                await self._process_learning(context)
            else:
                await self._process_other_context(context)
                
            # Mark as done
            self.context_queue.task_done()
    
    async def _process_message(self, context: Context) -> None:
        """Process a message context - to be implemented by subclasses"""
        raise NotImplementedError("Subclasses must implement _process_message")
    
    async def _process_task(self, context: Context) -> None:
        """Process a task context - to be implemented by subclasses"""
        raise NotImplementedError("Subclasses must implement _process_task")
    
    async def _process_command(self, context: Context) -> None:
        """Process a command context - to be implemented by subclasses"""
        raise NotImplementedError("Subclasses must implement _process_command")
    
    async def _process_learning(self, context: Context) -> None:
        """Process a learning context - to be implemented by subclasses"""
        raise NotImplementedError("Subclasses must implement _process_learning")
    
    async def _process_other_context(self, context: Context) -> None:
        """Process other context types - to be implemented by subclasses"""
        raise NotImplementedError("Subclasses must implement _process_other_context")
    
    async def generate_thought(self, thought_type: ThoughtType, content_prompt: str) -> Thought:
        """Generate an internal thought using the agent's LLM"""
        # To be implemented with actual LLM generation in subclasses
        raise NotImplementedError("Subclasses must implement generate_thought")
    
    async def shutdown(self) -> None:
        """Cleanly shutdown the agent"""
        self.logger.info(f"Shutting down agent {self.agent_id}")
        self.status = AgentStatus.OFFLINE
        # Additional cleanup to be implemented by subclasses

# ====== Message Broker ======

class MessageBroker:
    """Handles communication between agents"""
    
    def __init__(self):
        self.agents: Dict[str, BaseAgent] = {}
        self.context_handlers: Dict[ContextType, List[str]] = {}
        self.logger = logging.getLogger("message_broker")
    
    def register_agent(self, agent: BaseAgent) -> None:
        """Register an agent with the broker"""
        self.agents[agent.agent_id] = agent
        
        # Register context subscriptions
        for context_type in agent.subscribed_contexts:
            if context_type not in self.context_handlers:
                self.context_handlers[context_type] = []
            self.context_handlers[context_type].append(agent.agent_id)
    
    async def publish_context(self, context: Context) -> None:
        """Publish a context to relevant agents"""
        self.logger.debug(f"Publishing context: {context.context_id}")
        
        target_agents = []
        
        # If specific targets are specified
        if context.target_agents:
            target_agents = [agent_id for agent_id in context.target_agents if agent_id in self.agents]
        # Otherwise, send to all subscribed agents
        else:
            target_agents = self.context_handlers.get(context.context_type, [])
        
        # Send to each target agent
        for agent_id in target_agents:
            if agent_id != context.source_agent:  # Don't send back to source
                await self.agents[agent_id].receive_context(context)